In [0]:
import pandas as pd

URI_TREINO = "https://github.com/tgcsantos/quaretenadados/blob/master/DADOS_TREINO.csv?raw=true"
URI_TESTE = "https://github.com/tgcsantos/quaretenadados/raw/master/DADOS_TESTE.csv"
URI_DESAFIOQT = "https://github.com/tgcsantos/quaretenadados/raw/master/DESAFIOQT.csv"

dados_treino = pd.read_csv(URI_TREINO)
dados_teste = pd.read_csv(URI_TESTE)
dados_desafioqt = pd.read_csv(URI_DESAFIOQT)

erro_treino = "Erro ao carregar dados de treino"
erro_teste = "Erro ao carregar dados de teste"
erro_desafioqt = "Erro ao carregar dados de submissão"

assert dados_treino.shape == (150000, 5), erro_treino
assert dados_teste.shape == (20000, 5), erro_teste
assert dados_desafioqt.shape == (10000, 5), erro_desafioqt

In [0]:
coluna_label = 'NU_NOTA_LC'
coluna_features = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

X_treino = dados_treino[coluna_features].to_numpy()
Y_treino = dados_treino[coluna_label].to_numpy()
X_teste = dados_teste[coluna_features].to_numpy()
Y_teste = dados_teste[coluna_label].to_numpy()

In [0]:
#Exemplo de classificação com Dummy
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error

modelo_dummy = DummyRegressor()
modelo_dummy.fit(X_treino, Y_treino)
dummy_predicoes = modelo_dummy.predict(X_teste)

avaliacao_dummy = mean_squared_error(Y_teste, dummy_predicoes)

print(f"Minha avaliação nos dados de teste foi de {avaliacao_dummy}")

Minha avaliação nos dados de teste foi de 5219.286870186777


In [0]:
#aplicando o pre-processamento
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_treino)  # Don't cheat - fit only on training data
X_treino = scaler.transform(X_treino)
X_teste = scaler.transform(X_teste)  # apply same transformation to test data

In [0]:
from sklearn.linear_model import ElasticNetCV
modelo11 = ElasticNetCV(cv=10, random_state=0, verbose=True, l1_ratio=1, n_alphas=1000)

In [0]:
#melhor até agora 2045.6435210824218
#melhor até agora alpha=0.9, learning_rate=0.1, n_estimators=150

#GradientBoostingRegressor(alpha=0.1, ccp_alpha=0.0, criterion='friedman_mse',
#                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
#                          max_features=None, max_leaf_nodes=None,
#                          min_impurity_decrease=0.0, min_impurity_split=None,
#                          min_samples_leaf=1, min_samples_split=2,
#                          min_weight_fraction_leaf=0.0, n_estimators=100,
#                          n_iter_no_change=None, presort='deprecated',
#                          random_state=None, subsample=1.0, tol=0.0001,
#                         validation_fraction=0.1, verbose=0, warm_start=False)
from sklearn.ensemble import GradientBoostingRegressor
modelo21 = GradientBoostingRegressor(alpha=0.9, learning_rate=0.1, n_estimators=150, random_state=0)
modelo21.fit(X_treino, Y_treino)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=150,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=0, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
predicoes_modelo21= modelo21.predict(X_teste)
avaliacao21 = mean_squared_error(Y_teste, predicoes_modelo21)
print(f"Minha avaliação nos dados de teste foi de {avaliacao21}")
print(modelo21.score(X_teste, Y_teste) * 100)
print(predicoes_modelo21[:5].round(1))
print(Y_teste[:5])

Minha avaliação nos dados de teste foi de 2045.590495308854
60.804922984479816
[529.3 458.9 649.7 514.2 553.1]
[550.  406.3 652.3 469.1 580.5]


In [0]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
modelo = HistGradientBoostingRegressor(random_state=0, learning_rate=0.1, max_iter=1000)
modelo.fit(X_treino, Y_treino)
predicoes_modelo= modelo.predict(X_teste)
avaliacao = mean_squared_error(Y_teste, predicoes_modelo)
print(f"Minha avaliação nos dados de teste foi de {avaliacao}")
print(modelo21.score(X_teste, Y_teste) * 100)
print(predicoes_modelo[:5].round(1))
print(Y_teste[:5])

Minha avaliação nos dados de teste foi de 2108.779315426458
60.804922984479816
[534.7 447.6 656.8 510.9 560.5]
[550.  406.3 652.3 469.1 580.5]


In [0]:
#sem o scaler = 2046.5932432530724
from sklearn import linear_model
from sklearn.ensemble import BaggingRegressor
modelo = BaggingRegressor(base_estimator=GradientBoostingRegressor(alpha=0.9, learning_rate=0.1, n_estimators=150, random_state=0), n_estimators=20)
modelo.fit(X_treino, Y_treino)
predicoes_modelo= modelo.predict(X_teste)
avaliacao = mean_squared_error(Y_teste, predicoes_modelo)
print(f"Minha avaliação nos dados de teste foi de {avaliacao}")
print(modelo.score(X_teste, Y_teste) * 100)
print(predicoes_modelo[:5].round(1))
print(Y_teste[:5])




Minha avaliação nos dados de teste foi de 2047.0174250879518
60.777581919532686
[528.6 459.5 649.  514.9 554.1]
[550.  406.3 652.3 469.1 580.5]


In [0]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import SGDRegressor
modelo = AdaBoostRegressor(base_estimator=SGDRegressor(alpha=0.01,random_state=1, learning_rate='adaptive', warm_start=True, tol=1e-7, eta0=0.000001))
modelo.fit(X_treino, Y_treino)
predicoes_modelo= modelo.predict(X_teste)
avaliacao = mean_squared_error(Y_teste, predicoes_modelo)
print(f"Minha avaliação nos dados de teste foi de {avaliacao}")
print(modelo.score(X_teste, Y_teste) * 100)
print(predicoes_modelo[:5].round(1))
print(Y_teste[:5])

Minha avaliação nos dados de teste foi de 2192.731544595781
57.98558804321397
[533.3 446.4 651.4 510.9 547.9]
[550.  406.3 652.3 469.1 580.5]


In [0]:
#valor a bater 2044.5002402462057
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LassoCV, BayesianRidge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import SGDRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import GradientBoostingRegressor

estimators = [
    ('Random Forest', RandomForestRegressor(random_state=0)),
    ('Lasso', LassoCV()),
    ('Gradient Boosting', HistGradientBoostingRegressor(random_state=0)),
    ('Gradient', GradientBoostingRegressor(alpha=0.9, learning_rate=0.1, n_estimators=150, random_state=0)),
    ('ELastic',ElasticNetCV()),
    ('SGD', SGDRegressor(alpha=0.01,random_state=1, learning_rate='adaptive', warm_start=True, tol=1e-7, eta0=0.000001)),
    ('LinearSVR', LinearSVR(C=1)),
    ('Bayesian', BayesianRidge(verbose=True))]

stacking_regressor = StackingRegressor(
    estimators=estimators, final_estimator=RidgeCV()
)
modelo = stacking_regressor
modelo.fit(X_treino, Y_treino)
predicoes_modelo= modelo.predict(X_teste)
avaliacao = mean_squared_error(Y_teste, predicoes_modelo)
print(f"Minha avaliação nos dados de teste foi de {avaliacao}")
print(modelo.score(X_teste, Y_teste) * 100)
print(predicoes_modelo[:5].round(1))
print(Y_teste[:5])

Convergence after  2  iterations
Convergence after  2  iterations
Convergence after  2  iterations
Convergence after  2  iterations
Convergence after  2  iterations
Convergence after  2  iterations
Minha avaliação nos dados de teste foi de 2044.4242220659812
60.827269671013795
[529.7 458.2 649.7 513.7 553.4]
[550.  406.3 652.3 469.1 580.5]
